In [1]:
%load_ext lab_black

# AutoUi

With ipyautoui we can create ipywidgets from either a **json-schema** or a **pydantic model**. This makes it quick and easy to whip up a user interface when required.

In [2]:
from ipyautoui import AutoUi
import json
from pydantic import BaseModel, Field
from ipyautoui.constants import DIR_MODULE
from ipyautoui._utils import display_pydantic_json
import ipyautoui

print(ipyautoui.__doc__)


ipyautoui is used to quickly and efficiently create ipywidgets from pydantic schema.

The module has the capability to take a pydantic schema and create a ipywidget from that schema.
The main features being that you can produce a widget from many field types and also save the
data as a JSON easily.

ipyautoui is designed to be extensible. 
all widgets, as a minimum must satisfy the following criteria: 
- be initiated by the following keyword arguments (in addition to others as required):
    - schema
    - value
- the schema must be a valid jsonschema and where possible use the terms defined in jsonschema

Example::

    from ipyautoui.constants import DISPLAY_AUTOUI_SCHEMA_EXAMPLE
    DISPLAY_AUTOUI_SCHEMA_EXAMPLE()
    



### Creating Simple Widget

So let's create a simple pydantic class. Here we have one text field.

In [3]:
# create a pydantic model (or a json-schema) defining the fields of interest
class AutoUiExample(BaseModel):
    text: str = Field(default="Test", description="This description is very important")


data = {"text": "this is a value"}
ui = AutoUi(schema=AutoUiExample, value=data)
display(ui)

AutoUi(children=(VBox(children=(HBox(), HBox(children=(ToggleButton(value=False, icon='code', layout=Layout(di…

In [4]:
ui.schema

{'title': 'AutoUiExample',
 'type': 'object',
 'properties': {'text': {'title': 'Text',
   'description': 'This description is very important',
   'default': 'Test',
   'type': 'string'}}}

### Writing to JSON

Let's define the save location.

In [5]:
import pathlib

save_path = pathlib.Path(".") / "test.simpleaui.json"
print(f"Save Location is: {save_path}")

Save Location is: test.simpleaui.json


In [6]:
ui.file(path=save_path)
AutoUiRenderer = AutoUi.create_autoui_renderer(
    schema=AutoUiExample, fn_onsave=lambda: print("done")
)
ui_simple = AutoUiRenderer(path=save_path)
display(ui_simple)

AutoRenderer(children=(VBox(children=(HBox(children=(SaveButtonBar(children=(ToggleButton(value=False, button_…

In [7]:
import ipywidgets as widgets

ui_simple.autowidget.insert_rows = {0: widgets.Button()}

### A more Complex Example Model

AutoUiRenderer TestAutoLogicLet's look at a complete pydantic model producing all of the possible widgets. 
within `import ipyautoui.test_schema` there is a class called `TestAutoLogic` that outlines what is possible. 
Explore the python file below.

In [8]:
from ipyautoui.test_schema import TestAutoLogic  # the schema shown in the file above
import ipywidgets as widgets
from ipyautoui._utils import display_python_module
from IPython.display import Markdown

DIR_EXAMPLE = pathlib.Path(".")

save_path = DIR_EXAMPLE / "test.complexaui.json"
TestAutoLogic().file(path=save_path)

AutoUiRenderer = AutoUi.create_autoui_renderer(
    schema=TestAutoLogic, fn_onsave=lambda: print("this isnt working")
)
ui_complex = AutoUiRenderer(path=save_path)
ui_complex.file(path=save_path)


s_py = display_python_module(ipyautoui.test_schema, show=False, return_str=True)
out_py = widgets.Output()
with out_py:
    display(Markdown(s_py))

s_sch = f"""
```json
{json.dumps(TestAutoLogic.schema(), indent=4)}
```
"""
out_sch = widgets.Output()
with out_sch:
    display(Markdown(s_sch))


s_js = f"""
```json
{json.dumps(ui_complex.value, indent=4)}
```
"""
out_json = widgets.Output()
with out_json:
    display(Markdown(s_js))

tab = widgets.Tab([ui_complex, out_py, out_sch, out_json])
titles = {0: "AutoUi", 1: "pydantic-model", 2: "jsonschema", 3: "autoui value"}
tab._titles = titles
tab

In [9]:
class DataFrameCols(BaseModel):
    string: str = Field("string", aui_column_width=100)
    integer: int = Field(1, aui_column_width=80)
    floater: float = Field(3.1415, aui_column_width=70, aui_sig_fig=3)
    something_else: float = Field(324, aui_column_width=100)

In [10]:
import typing


class DataFrameCols(BaseModel):
    string: str = Field("string", aui_column_width=100)
    integer: int = Field(1, aui_column_width=80)
    floater: float = Field(3.1415, aui_column_width=70, aui_sig_fig=3)
    something_else: float = Field(324, aui_column_width=100)


class TestDataFrame(BaseModel):
    """a description of TestDataFrame"""

    dataframe: typing.List[DataFrameCols] = Field(
        default_factory=lambda: [], format="dataframe"
    )

In [11]:
auto_grid = AutoUi(schema=TestDataFrame)
display(auto_grid)

AutoUi(children=(VBox(children=(HBox(), HBox(children=(ToggleButton(value=False, icon='code', layout=Layout(di…

In [12]:
auto_grid.value = {
    "dataframe": [
        {
            "string": "important string",
            "integer": 1,
            "floater": 3.14,
            "something_else": 324,
        },
        {"string": "update", "integer": 4, "floater": 3.12344, "something_else": 123},
        {"string": "evening", "integer": 5, "floater": 3.14, "something_else": 235},
        {"string": "morning", "integer": 5, "floater": 3.14, "something_else": 12},
        {"string": "number", "integer": 3, "floater": 3.14, "something_else": 123},
    ]
}